# Intelligent Fabric Monitoring with Smart Detection

This notebook demonstrates the new **intelligent monitoring capabilities** that automatically detect Microsoft's workspace monitoring and adjust collection strategy to avoid conflicts while maximizing value.

## ✨ What's New

- **🧠 Smart Detection**: Automatically detects Microsoft's workspace monitoring
- **🎯 Conflict Avoidance**: Skips data collection that conflicts with official monitoring
- **⚡ Unique Value Focus**: Prioritizes data not covered by Microsoft
- **🔧 Flexible Strategies**: Choose from auto, full, complement, or minimal approaches

## 📋 Prerequisites

1. **FabricLA-Connector v1.0+** with intelligent monitoring
2. **Azure Log Analytics workspace** with DCR configured
3. **Fabric authentication** (Azure CLI or service principal)
4. **Workspace permissions** to read monitoring status

## 🔧 Configuration & Setup

In [ ]:
# Install the latest FabricLA-Connector if needed
# %pip install fabricla-connector --upgrade

import os
from fabricla_connector import (
    run_intelligent_monitoring_cycle,
    check_workspace_monitoring_status,
    get_collection_recommendations,
    print_monitoring_status,
    get_monitoring_strategy,
    get_fabric_config,
    print_config_status
)

print("✅ FabricLA-Connector with Intelligent Monitoring loaded")

In [ ]:
# Configuration - Update these with your values
WORKSPACE_ID = "your-workspace-id"  # Replace with your workspace ID
CAPACITY_ID = "your-capacity-id"    # Replace with your capacity ID (optional)

# Optional: Set monitoring strategy via environment variable
# Options: 'auto' (default), 'full', 'complement', 'minimal'
os.environ['FABRIC_MONITORING_STRATEGY'] = 'auto'

print(f"🎯 Target Workspace: {WORKSPACE_ID}")
print(f"⚡ Target Capacity: {CAPACITY_ID or 'Not configured'}")
print(f"🧠 Strategy: {os.environ.get('FABRIC_MONITORING_STRATEGY', 'auto')}")

## 🔍 Step 1: Check Configuration Status

In [ ]:
# Verify configuration is properly set up
print_config_status()

## 🔍 Step 2: Detect Workspace Monitoring Status

In [ ]:
# Check if Microsoft's workspace monitoring is enabled
monitoring_status = check_workspace_monitoring_status(WORKSPACE_ID)

print("🔍 Workspace Monitoring Detection Results:")
print("=" * 50)

enabled = monitoring_status.get('workspace_monitoring_enabled')
method = monitoring_status.get('detection_method', 'unknown')

if enabled is True:
    print(f"✅ Microsoft Workspace Monitoring: ENABLED")
    print(f"📊 Detection Method: {method}")
    if monitoring_status.get('eventhouse_id'):
        print(f"🏠 Eventhouse ID: {monitoring_status['eventhouse_id']}")
elif enabled is False:
    print(f"❌ Microsoft Workspace Monitoring: DISABLED")
    print(f"📊 Detection Method: {method}")
else:
    print(f"⚠️ Microsoft Workspace Monitoring: UNKNOWN")
    print(f"📊 Detection Method: {method}")
    if monitoring_status.get('error'):
        print(f"❌ Error: {monitoring_status['error']}")

print("\n💡 This detection determines which data sources we should collect to avoid conflicts.")

## 📋 Step 3: Get Collection Recommendations

In [ ]:
# Get intelligent recommendations for what to collect
recommendations = get_collection_recommendations(WORKSPACE_ID, strategy='auto')

print("📋 Intelligent Collection Recommendations:")
print("=" * 50)

if 'error' in recommendations:
    print(f"❌ Error getting recommendations: {recommendations['error']}")
else:
    for source, rec in recommendations['recommendations'].items():
        status = "✅ COLLECT" if rec['collect'] else "❌ SKIP"
        reason = rec.get('reason', 'Unknown reason')
        
        print(f"{status} {source.replace('_', ' ').title()}")
        print(f"   └─ {reason}")
        
        if not rec['collect'] and rec.get('alternative'):
            print(f"   💡 Alternative: {rec['alternative']}")
        print()

## 🚀 Step 4: Run Intelligent Monitoring

This is the main intelligent monitoring cycle that automatically adapts to your workspace monitoring setup.

In [ ]:
# Run the intelligent monitoring cycle
print("🚀 Starting Intelligent Monitoring Cycle...")
print("=" * 50)

result = run_intelligent_monitoring_cycle(
    workspace_id=WORKSPACE_ID,
    capacity_id=CAPACITY_ID,  # Can be None if not available
    strategy_override=None    # Uses environment variable or 'auto'
)

print("\n" + "=" * 50)
print("📊 MONITORING CYCLE RESULTS")
print("=" * 50)

if result.get('status') == 'success':
    summary = result['summary']
    print(f"✅ Status: SUCCESS")
    print(f"📊 Data Sources Collected: {summary['collected_sources']}/{summary['total_sources']}")
    print(f"📝 Total Records Ingested: {summary['total_records']:,}")
    print(f"🧠 Strategy Used: {result['strategy']}")
    
    if result['collections']:
        print(f"\n✅ Successful Collections:")
        for source, details in result['collections'].items():
            records = details.get('records_collected', 0)
            print(f"   • {source.replace('_', ' ').title()}: {records:,} records")
    
    if result['skipped_collections']:
        print(f"\n⏭️ Skipped Collections:")
        for source, decision in result['skipped_collections'].items():
            reason = decision.get('reason', 'Unknown')
            print(f"   • {source.replace('_', ' ').title()}: {reason}")
            if decision.get('alternative'):
                print(f"     💡 Alternative: {decision['alternative']}")
else:
    print(f"❌ Status: ERROR")
    print(f"❌ Error: {result.get('error', 'Unknown error')}")

## 🔧 Step 5: Try Different Strategies

Experiment with different monitoring strategies to see how they affect collection decisions.

In [ ]:
# Compare different strategies
strategies = ['auto', 'full', 'complement', 'minimal']

print("🔧 Strategy Comparison:")
print("=" * 60)

for strategy in strategies:
    print(f"\n🧠 Strategy: {strategy.upper()}")
    print("-" * 30)
    
    strategy_recs = get_collection_recommendations(WORKSPACE_ID, strategy=strategy)
    
    if 'error' not in strategy_recs:
        collect_count = sum(1 for rec in strategy_recs['recommendations'].values() if rec['collect'])
        total_count = len(strategy_recs['recommendations'])
        
        print(f"📊 Will collect: {collect_count}/{total_count} data sources")
        
        for source, rec in strategy_recs['recommendations'].items():
            status = "✅" if rec['collect'] else "❌"
            print(f"   {status} {source.replace('_', ' ').title()}")
    else:
        print(f"❌ Error: {strategy_recs['error']}")

## 🎯 Step 6: Complementary Monitoring (Recommended)

If you want to only collect data that doesn't conflict with Microsoft's workspace monitoring, use the complementary strategy.

In [ ]:
# Import the complementary monitoring function
from fabricla_connector import run_complementary_monitoring_cycle

print("🎯 Running Complementary Monitoring (Conflict-Free)...")
print("=" * 50)

complementary_result = run_complementary_monitoring_cycle(
    workspace_id=WORKSPACE_ID,
    capacity_id=CAPACITY_ID
)

if complementary_result.get('status') == 'success':
    summary = complementary_result['summary']
    print(f"✅ Complementary monitoring completed successfully")
    print(f"📊 Unique data sources collected: {summary['collected_sources']}")
    print(f"📝 Total records: {summary['total_records']:,}")
    print(f"\n💡 This strategy focuses only on data NOT covered by Microsoft's workspace monitoring.")
else:
    print(f"❌ Error: {complementary_result.get('error')}")

## 📚 Understanding the Strategies

### 🧠 **Auto Strategy (Recommended)**
- Automatically detects workspace monitoring status
- Collects conflicting data only if workspace monitoring is disabled
- Focuses on unique value when workspace monitoring is enabled

### 🎯 **Complement Strategy**
- Only collects data NOT covered by Microsoft's workspace monitoring
- Safest approach to avoid any conflicts
- Recommended for customers using Microsoft's official monitoring

### 🔧 **Full Strategy**
- Collects all data regardless of workspace monitoring status
- May duplicate Microsoft's data but provides complete coverage
- Useful for customers who prefer centralized monitoring in Log Analytics

### ⚡ **Minimal Strategy**
- Only collects critical unique data (pipelines, capacity)
- Smallest footprint, highest unique value
- Good for cost-conscious deployments

## 🚀 Next Steps

1. **Set up automation**: Use the intelligent workflows in your scheduled notebooks
2. **Monitor Log Analytics**: Query your custom tables for insights
3. **Configure alerts**: Set up alerts on the collected operational data
4. **Complement Microsoft's monitoring**: Use alongside workspace monitoring for complete coverage

## 📖 Documentation

For more information, see the [FabricLA-Connector documentation](https://github.com/Keayoub/FabricLA-Connector) with the latest intelligent monitoring features.